# 設定ファイルの編集
---

OpenHPC環境の設定ファイルの内容を変更します。

## 概要

このNotebookではVCノードの設定ファイルを編集し、その結果を各VCノード配布する手順を示します。

![処理の流れ](images/ohpc-011.png)

主な手順は以下のようになります。

1. マスターノードの設定ファイルをNotebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して設定ファイルの変更をおこなう
4. 変更した設定ファイルを各VCノードに配置する

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* 各VCノードに対してAnsibleで操作できるように設定されている
* 各VCノードに対して管理者権限で操作が行える

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

操作対象となる UnitGroup 名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

疎通確認を行います。

In [ ]:
!ansible {ugroup_name} -m ping 

管理者権限でコマンドが実行できることを確認します。

In [ ]:
!ansible {ugroup_name} -b -a 'whoami'

## パラメータの設定

### 編集対象となるファイルの指定

対象となるファイルのパスを指定してください。

In [ ]:
# (例)
# target_file = '/etc/slurm/slurm.conf'

target_file = 

## 設定ファイルの編集

OpenHPC環境の設定ファイルを取得して、Jupyter Notebookの編集機能を用いて設定ファイルを編集します。

次のセルを実行すると、以下の手順が実行されます。

1. マスターノードの設定ファイルをローカル環境に取得する
2. 取得した設定ファイルのバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

表示されたリンクをクリックして、設定ファイルを編集してください。

ローカル環境に取得したファイルは、以下のディレクトリに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは同じディレクトリに、設定ファイルのサフィックスに `.orig` を付けたファイル名で格納されます。

In [ ]:
%run scripts/edit_conf.py
fetch_conf(target_file, ugroup_name, 'master')

上のセルに表示されているリンクをクリックするとファイルの編集画面が開きます。編集を行い、その結果をファイルに保存してください。ファイルの保存を行うにはメニューの File -- Save を選択するか、キーボードで Ctrl-S をタイプしてください。

編集後に次のセルを実行すると、編集の前後での差分を確認することができます。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(target_file, ugroup_name, 'master')

## 編集した設定ファイルの反映

編集したファイルをVCノードに配置して、設定ファイルの変更内容をコンテナに反映させます。

### 編集内容の反映

前章で編集した設定ファイルを各VCノードに配置します。

次のセルを実行すると、以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイルを各VCノードに配置する

In [ ]:
%run scripts/edit_conf.py
upload_conf(target_file, ugroup_name)

各ノードの設定ファイルが変更されましたので、必要に応じてサービスの再起動や設定ファイルの再読み込みなどの操作を行なってください。例えばSLURMデーモンに設定ファイルの再読み込みを指示するには次のセルのコメントを外して実行してください。

In [ ]:
# !ansible {ugroup_name}_master -b -a 'scontrol reconfigure'